We want to do a Hawkins+2017 style run, with $\varpi_{\rm zp}$ having a highly informed prior, and the RC parameters being free.

In [1]:
# !/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
sns.set_palette('colorblind')
matplotlib.rc('xtick', labelsize=15)
matplotlib.rc('ytick', labelsize=15)
matplotlib.rc('axes',labelsize=15)

import pandas as pd
import pystan
import corner

import pickle
import glob

import os
import sys

__outdir__ = os.path.expanduser('~')+'/PhD/Gaia_Project/Output/Parallax_Runs/Highruns/'
__datdir__ = os.path.expanduser('~')+'/PhD/Gaia_Project/data/KepxDR2/'

In [2]:
overwrite = True
hawkstan = '''
functions {
    real bailerjones_lpdf(real r, real L){
        return log((1/(2*L^3)) * (r*r) * exp(-r/L));
    }
    real precalc_multinormal_lpdf(vector oo, vector oo_true, real logdetc, matrix invc, int N, real Nfloat){
        vector[N] r;
        r = oo - oo_true;

        return -0.5 * ((r' * invc * r) + logdetc + Nfloat * log(2*pi()));
    }
}
data {
    int<lower = 0> N;
    real<lower= 0> Nfloat;
    vector[N] m;
    vector<lower=0>[N] m_err;
    vector[N] oo;
    vector<lower=0>[N] RlEbv;

    matrix[N, N] invc;
    real logdetc;

    real oozp_init;
    real oozp_spread;
    real muH;
}
parameters {
    //Hyperparameters
    real mu;
    real<lower=0.> sigma;
    real<lower=1.> sigo;
    real<lower=0.5,upper=1.> Q;
    real<lower=.1, upper=4000.> L;
    real oo_zp;

    //Latent parameters
    vector[N] M_infd_std;
    vector[N] Ai;
    vector<lower = 1.>[N] r_infd;
}
transformed parameters{
    //Inferred and transformed parameters
    vector[N] M_infd;

    //Operations
    for (n in 1:N){
        M_infd[n] = mu + sigma * M_infd_std[n]; //Rescale the M fit
    }
}
model {
    //Define calculable properties
    vector[N] m_true;
    vector[N] oo_true;

    //Hyperparameters [p(theta_rc, L)]
    mu ~ normal(muH, 1.0); // Prior from seismo
    sigma ~ normal(0.0, 1.0);
    Q ~ normal(1., .25);
    sigo ~ normal(3.0, 1.0);
    L ~ uniform(0.1, 4000.);   // Prior on the length scale
    oo_zp ~ normal(oozp_init, oozp_spread); // Prior on the offset (in mu as)

    //Latent parameters [p(alpha_i | theta_rc, L)]
    Ai ~ normal(RlEbv, 0.05);
    for (n in 1:N){
        r_infd[n] ~ bailerjones(L);
        target += log_mix(Q,
            normal_lpdf(M_infd_std[n] | 0., 1.),
            normal_lpdf(M_infd_std[n] | 0., sigo));
    }

    //Calculable properties
    for (n in 1:N){
        m_true[n] = M_infd[n] + 5*log10(r_infd[n]) - 5 + Ai[n];
        oo_true[n] = (1000./r_infd[n]) + (oo_zp/1000.);
    }

    //Observables [p(D | theta_rc, L, alpha)]
    oo ~ precalc_multinormal(oo_true, logdetc, invc, N, Nfloat);
    m ~ normal(m_true, m_err); //Measurement uncertainty on magnitude
}

'''
model_path = 'hawkstan.pkl'
if overwrite:
    print('Updating Stan model')
    sm = pystan.StanModel(model_code = hawkstan, model_name='hawkstan')
    with open(model_path, 'wb') as f:
        pickle.dump(sm, f)
else:
    if os.path.isfile(model_path):
        print('Loading in existing Stan Model') 
        sm = pickle.load(open(model_path, 'rb'))
    else:
        print('Saving Stan Model') 
        sm = pystan.StanModel(model_code = hawkstan, model_name='hawkstan')
        with open(model_path, 'wb') as f:
            pickle.dump(sm, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL hawkstan_58ac0be2ad086bcaefda86eac9809b56 NOW.


Updating Stan model


In [3]:
def read_data():
    sfile = __datdir__+'rcxyu18.csv'
    df = pd.read_csv(sfile)
    return df

def kernel(ra, dec, sigma, p):
    '''
    p[0] : Offset
    p[1] : Exponential decay scale
    '''
    dr = np.deg2rad(dec)
    thetaij = np.sqrt((np.subtract.outer(ra, ra)*np.cos(0.5*np.add.outer(dr, dr)))**2 + np.subtract.outer(dec, dec)**2)
    cov = p[0] * np.exp(-thetaij / p[1])
    np.fill_diagonal(cov, np.diag(cov) + sigma**2)

    if not np.all(np.linalg.eigvals(cov) > 0):
        raise ValueError("The matrix isn't positive-definite for some reason!")
    return cov

def get_covmatrix(df):
    p = [285*10**-6, 14.]
    Sigma = kernel(df.ra.values, df.dec.values, df.parallax_error.values, p)
    invc = np.linalg.inv(Sigma)
    logdetc = np.linalg.slogdet(Sigma)[1] * np.linalg.slogdet(Sigma)[0]

    return Sigma, invc, logdetc

In [4]:
kdf = read_data()
from sklearn.utils import shuffle
df = shuffle(kdf, random_state=24601)[:1000].reset_index()

In [5]:
from omnitool.literature_values import hawkvals
oozps = pd.DataFrame(index=['Lindegren','Zinn','Riess','Sahlholdt','Stassun','Hawkins','Uninformed','Khan'],
                     columns=['mu','spread'])
oozps.loc['Lindegren']['mu'] = -29.
oozps.loc['Lindegren']['spread'] = 1.
oozps.loc['Zinn']['mu'] = -52.8
oozps.loc['Zinn']['spread'] = 3.4
oozps.loc['Riess']['mu'] = -46.
oozps.loc['Riess']['spread'] = 13.
oozps.loc['Sahlholdt']['mu'] = -35.
oozps.loc['Sahlholdt']['spread'] = 16.
oozps.loc['Stassun']['mu'] = -82.
oozps.loc['Stassun']['spread'] = 33.
oozps.loc['Hawkins']['mu'] = 0.
oozps.loc['Hawkins']['spread'] = 1.
oozps.loc['Uninformed']['mu'] = 0.
oozps.loc['Uninformed']['spread'] = 1000.
oozps.loc['Khan']['mu'] = -47.02
oozps.loc['Khan']['spread'] = 0.80

In [6]:
band = 'K'
if band == 'K':
    rlebv = df.Aks.values
    mband = df.Kmag.values
    merr = df.e_Kmag.values
elif band == 'GAIA':
    rlebv = df.Ebv.values * 2.740 #As per Casagrande & Vandenberg 2018b
    #Correct the Gaia G mags as per Casagrande & Vandenberg 2018b
    mband = np.ones(len(df)) * df.GAIAmag.values
    sel = (mband > 6.) & (mband < 16.5)
    mband[sel] = 0.0505 + 0.9966*mband[sel]
    merr = np.ones(len(mband)) * 10.e-3 #Setting precision to 10mmag by default

Sigma, invc, logdetc = get_covmatrix(df)

dat = {'N':len(df),
       'Nfloat':np.float(len(df)),
        'm': mband,
        'm_err': merr,
        'oo': df.parallax.values,
        'RlEbv': rlebv,
        'logdetc': logdetc,
        'invc': invc,
        'muH': hawkvals[band] }


init= {'mu': hawkvals[band],
        'sigma': 0.1,
        'Q': 0.95,
        'sigo': 4.,
        'L': 1000.,
        'r_infd':df.r_est,
        'Ai':rlebv}

pars = ['mu', 'sigma', 'Q', 'sigo', 'L', 'oo_zp']
verbose = [r'$\mu_{RC} (mag)$',r'$\sigma_{RC} (mag)$',r'$Q$', r'$\sigma_o (mag)$', r'$L (pc)$', r'$\varpi_{zp} (\mu as)$']

In [ ]:
for author in oozps.index.values:
    runlabel = __outdir__+band+'_'+author
    
    dat['oozp_init'] = oozps.loc[author]['mu']
    dat['oozp_spread'] = oozps.loc[author]['spread']
    init['oozp'] = oozps.loc[author]['mu']
    
    fit = sm.sampling(data = dat, iter = 1000, chains=4, seed = 24601,
                        init = [init for i in range(4)])
    chain = np.array([fit[label] for label in pars])
    np.savetxt(runlabel+'_chains.txt', chain)
    
    pardict = {label:np.median(fit[label]) for label in pars}
    pardict.update({label+'_std':np.std(fit[label]) for label in pars})
    pardict = pd.DataFrame.from_dict(pardict, orient='index').T
    pardict.to_csv(runlabel+'_pars.csv')
    
    s = fit.summary()
    rhat = s['summary'][:,-1]
    np.savetxt(runlabel+'_rhats.txt',rhat)
    
    corner.corner(chain.T, labels=verbose,quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize":12})
    plt.savefig(runlabel+'_corner.png')
    plt.close()

/home/oliver/.local/lib/python2.7/site-packages/matplotlib/font_manager.py:1331: UserWarning: findfont: Font family [u'serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default.
INFO:matplotlib.texmanager:No LaTeX-compatible font found for the serif font family in rcParams. Using default

In [ ]:
band = 'GAIA'
if band == 'K':
    rlebv = df.Aks.values
    mband = df.Kmag.values
    merr = df.e_Kmag.values
elif band == 'GAIA':
    rlebv = df.Ebv.values * 2.740 #As per Casagrande & Vandenberg 2018b
    #Correct the Gaia G mags as per Casagrande & Vandenberg 2018b
    mband = np.ones(len(df)) * df.GAIAmag.values
    sel = (mband > 6.) & (mband < 16.5)
    mband[sel] = 0.0505 + 0.9966*mband[sel]
    merr = np.ones(len(mband)) * 10.e-3 #Setting precision to 10mmag by default

Sigma, invc, logdetc = get_covmatrix(df)

dat = {'N':len(df),
       'Nfloat':np.float(len(df)),
        'm': mband,
        'm_err': merr,
        'oo': df.parallax.values,
        'RlEbv': rlebv,
        'logdetc': logdetc,
        'invc': invc,
        'muH': hawkvals[band] }


init= {'mu': hawkvals[band],
        'sigma': 0.1,
        'Q': 0.95,
        'sigo': 4.,
        'L': 1000.,
        'r_infd':df.r_est,
        'Ai':rlebv}

pars = ['mu', 'sigma', 'Q', 'sigo', 'L', 'oo_zp']
verbose = [r'$\mu_{RC} (mag)$',r'$\sigma_{RC} (mag)$',r'$Q$', r'$\sigma_o (mag)$', r'$L (pc)$', r'$\varpi_{zp} (\mu as)$']

In [ ]:
for author in oozps.index.values:
    runlabel = __outdir__+band+'_'+author
    
    dat['oozp_init'] = oozps.loc[author]['mu']
    dat['oozp_spread'] = oozps.loc[author]['spread']
    init['oozp'] = oozps.loc[author]['mu']
    
    fit = sm.sampling(data = dat, iter = 5000, chains=4, seed = 24601,
                        init = [init for i in range(4)])
    chain = np.array([fit[label] for label in pars])
    np.savetxt(runlabel+'_chains.txt', chain)
    
    pardict = {label:np.median(fit[label]) for label in pars}
    pardict.update({label+'_std':np.std(fit[label]) for label in pars})
    pardict = pd.DataFrame.from_dict(pardict, orient='index').T
    pardict.to_csv(runlabel+'_pars.csv')
    
    s = fit.summary()
    rhat = s['summary'][:,-1]
    np.savetxt(runlabel+'_rhats.txt',rhat)
    
    corner.corner(chain.T, labels=verbose,quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize":12})
    plt.savefig(runlabel+'_corner.png')
    plt.close()